EXERCISE OVERVIEW

In this exercise, we're going to provide an overview of a RAG pipeline.

Prior to starting your will want to make sure your relational database tables AND your vector store tables have been created.  
If you need to review them scripts they are:  
    -*create_db_python.py*  
    -*create_pg_vector_table.py*  

[Click here to see lame_db schema](../static/images/mmr_db_schema.png)


Steps we're going to do in this exercise:  

Step 1: Use python docx library to load our document (any .docx file).  
Step 2: Chunking a document into paragraphs.  
Step 3: (First time only) Check env variables and ensure relational AND vector store tables have been created.  
Step 4: Connect to the database.  
Step 5: Store our paragraph chunks in the relational table (text_chunks).  
Step 6: After everything is stored as text in the database, we'll retrieve those paragraphs and embed/vectorize each one, storing it in the vector store (pgvector from Postgres in our case).  
Note: Later we'll learn about summarizing and text splitting but for now, we're just focused on the flow / "pipeline".   
Step 7: Next, we'll act as a user and send in a query, that will also be vectorized by the same tool we used to vectorize our paragraphs.  
Step 8: Then we'll find and retrieve the vectors that are most similar to our query.    
Step 9: Once we have the similar vectors, we'll submit them, along with the user query to our LLM and print our response.  
     

NOTE: When you setup directories for labs, you may have to run this code to add the virtual environment as a kernel (things work without this, but you get some warnings).  
*python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"*  


Step 1 - Import docx library for Python and instantiate a document object.

In [72]:
from docx import Document as DocxDocument
file_path = "Jupyter_Notebook_Info.docx"
doc = DocxDocument(file_path)

Step 2 - Extract and chunk into paragraphs.

In [73]:
doc_chunks = []
for para in doc.paragraphs:
            if para.text.strip():
                doc_chunks.append(para.text)

print(f"The number of document chunks is:{len(doc_chunks)}")

The number of document chunks is:45


Step 3 (First time only)

You will want to make sure your env file has the proper database configuration info.

If you have not yet ran *create_db_python.py*.  Take a minute to browse it and then run it please.
This creates your relational database tables.

The run *create_pgvector_table.py*.  This, creates the vector store table, mmr_vector.



Step 4 - Connect to the database using variables from .env file.

In [74]:
import psycopg2
import os
def get_connection():
    try:
        conn = psycopg2.connect(
                    dbname=os.getenv("dbname"),
                    user=os.getenv("dbuser"),
                    password=os.getenv("dbpassword"),
                    host=os.getenv("dbhost"),
                    port=os.getenv("dbport"),
                )
    except (psycopg2.DatabaseError, Exception) as error:
        print(f"Error: {error}")
    
    return conn



Step 5 - Store the chunks in the database.

In [75]:
from psycopg2 import sql
from pathlib import Path

try:

    # parse file extension
    file = Path(file_path).name
    suffix = Path(file_path).suffix
    file_ext = suffix.lstrip(".")

    # read in the entire file contents so we can store them
    with(open(file_path, "rb") as file_obj):
        file_data = file_obj.read()

    conn = get_connection()
    with conn.cursor() as cursor:
        # Insert file content into the complete_files table
       
        cursor.execute(
                sql.SQL(
                    "INSERT INTO complete_files (file_name, file_data, file_ext) VALUES (%s, %s, %s) RETURNING pk"
                ),
                [file, file_data, file_ext],
            )
            
        file_id = cursor.fetchone()[0]  # Get the generated primary key (pk)
        print(f"file_id is:{file_id}")

        for chunk in doc_chunks:   
            cursor.execute(
                sql.SQL("INSERT INTO text_chunks (text, file_id) VALUES (%s, %s) RETURNING pk"),
                [chunk, file_id],
            )
    
            conn.commit()  # Commit the transaction after each insert
except Exception as e:
    print(f"Error inserting file with chunks: {e}")
finally:
        if conn:
            conn.close()

file_id is:21


Step 6 - Vectorization  (Note: can take ~30-45 seconds to run to completion)
A. Retrieve Current Chunks from Database.  
B. Create embedding of each chunk.  
C. Store vector of embedding in pgvector (vector store).  

In [82]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
# Retreive Currrent Chunks from Database
current_chunks = []
conn = get_connection()
with conn.cursor() as cursor:
    cursor.execute(
        sql.SQL("SELECT pk, text FROM text_chunks WHERE is_vectorized = FALSE"),
    )

    rows = cursor.fetchall()
    print(f"len of rows:{len(rows)}")
    for row in rows:
         current_chunks.append(row) # append tuple of pk, text, file_id
    #     cursor.execute(f"UPDATE text_chunks SET is_vectorized = TRUE WHERE pk = %s",
    #                    (row[0],))
    
    conn.commit()

# Get the embedding model
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_key)
   
vector_dict = {}
for chunk in current_chunks:
     content = openai_embedding.embed_query(chunk[1])
     # Convert the embedding values to floats (ensures compatibility with storage formats)
     float_content = [float(x) for x in content]
     vector_dict[chunk[0]] = float_content


# # add the vectorized content to the vector store
with conn.cursor() as cursor:
    chunk_type = "text"  # Only working with text right now, so hard-coding chunk_type = "text"
    
    for cid, vec in vector_dict.items():
        cursor.execute(
            sql.SQL("INSERT INTO mmr_vector (vector, chunk_type, chunk_id) VALUES (%s, %s, %s)"),
            [vec, chunk_type, cid]
        )

conn.commit()

Step 7 - Vectorize Incoming Query.

In [83]:

query = "Where should I point my web browser after Jupyter is running?"
# query = "What is the command to install jupyter?"
# query = "You can execute a cell by clicking on it and pressing what?"
# query = "What City and State had the highest temperature?"

vectorized_query = openai_embedding.embed_query(query)

Step 8 - Find Similar Vectors

pgvector similarity search operators:  
<->:
Represents the Euclidean distance between two vectors, which is the "straight-line" distance between them in multi-dimensional space.  
<=>:
Calculates the cosine similarity between vectors, which is often preferred for high-dimensional data as it focuses on the angle between vectors rather than their magnitude.  
<#>
: Computes the inner product of two vectors, where each corresponding element is multiplied and summed.  

In [85]:

top_k = 3 # only bring back the top 3 results
conn = get_connection()
with conn.cursor() as cur:
    cur.execute(
        sql.SQL(
            """SELECT pk, chunk_type, chunk_id, 1 - (vector <=> %s::VECTOR) AS similarity
               FROM mmr_vector
               ORDER BY similarity DESC
               LIMIT %s"""
        ),
        [vectorized_query, top_k],
    )
    rows = cur.fetchall()
    similar_chunk_ids = []
    if rows:
        for row in rows:
            similar_chunk_ids.append(row[2])
    else:
        print("No results found.")


      

Step 9 - Get the text chunks of the closest matches.

In [86]:
conn = get_connection()
with conn.cursor() as cur:
    print(f"similar_chunk_ids:{similar_chunk_ids}")
    similar_context = []
    for chunk_id in similar_chunk_ids:
        cur.execute(
            sql.SQL("""SELECT text FROM text_chunks where pk = %s"""),
            [chunk_id],
        )
        row = cur.fetchone()  # Fetch only one row for the current chunk_id
        similar_context.append(row[0])



similar_chunk_ids:[303, 303, 304]


Step 10 - Retrieval  
Submit Similar Vectors to LLM with query to retrieve a response.

Note the prompt structure.  

In [87]:
from openai import OpenAI
from pprint import pprint

# Show the similar content retrieved
for sc in similar_context:
    pprint(f"CONTEXT ITEM:{sc}")

# Format the prompt
prompt = f"""You are an assistant for question-answering tasks. Use only 
the following pieces of retrieved context to answer the 
question. Use 3 sentences maximum to keep your answer concise. Here's a query: 
{query} and here are similar queries of retrieved context: {similar_context}. Again,
only base your answer on the similar queries data within the similar context."""

# Call the OpenAI ChatCompletion API using the updated method
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": query},
    ],
)

# Extract and print the response
print("\n\n")
pprint(f"RESPONSE:{response.choices[0].message.content.strip()}")

'CONTEXT ITEM:Jupyter Notebook Setup'
'CONTEXT ITEM:Jupyter Notebook Setup'
('CONTEXT ITEM:Once Jupyter is running, point your web browser at '
 'http://localhost:8888 to start using Jupyter notebooks. If everything worked '
 'correctly, you should see a screen like this, showing all available Jupyter '
 'notebooks in the current directory:')



('RESPONSE:The retrieved context does not provide information on the command '
 'to install Jupyter.')
